In [1]:
import json
from datetime import datetime

# global list to store all sales records
sales_records = []

def load_data_from_file():
    """load existing sales records from json file"""
    global sales_records
    try:
        with open('sales_data.json', 'r') as file:
            sales_records = json.load(file)
            print("\nprevious sales data loaded successfully!")
    except FileNotFoundError:
        print("\nno previous sales data found. starting fresh!")
        sales_records = []
    except json.JSONDecodeError:
        print("\ndata file corrupted. starting with empty records.")
        sales_records = []

def save_data_to_file():
    """save all sales records to json file"""
    try:
        with open('sales_data.json', 'w') as file:
            json.dump(sales_records, file, indent=4)
            print("\nsales data saved successfully!")
    except Exception as e:
        print(f"\nerror saving data: {e}")

def add_sale_record():
    """add a new sale transaction"""
    print("\n" + "="*50)
    print("add new sale")
    print("="*50)
    
    product_name = input("\nproduct name: ").strip()
    
    if not product_name:
        print("product name cannot be empty!")
        return
    
    # get quantity with validation
    while True:
        try:
            quantity = int(input("quantity sold: "))
            if quantity > 0:
                break
            else:
                print("quantity must be greater than 0!")
        except ValueError:
            print("invalid input! enter a valid number.")
    
    # get price per unit with validation
    while True:
        try:
            price_per_unit = float(input("price per unit (in rupees): "))
            if price_per_unit > 0:
                break
            else:
                print("price must be greater than 0!")
        except ValueError:
            print("invalid input! enter a valid price.")
    
    customer_name = input("customer name: ").strip()
    payment_method = input("payment method (cash/card/upi/other): ").strip().lower()
    notes = input("additional notes (optional): ").strip()
    
    # calculate total amount
    total_amount = quantity * price_per_unit
    
    # create sale record dictionary
    sale_entry = {
        'sale_id': len(sales_records) + 1,
        'product': product_name.title(),
        'quantity': quantity,
        'price_per_unit': price_per_unit,
        'total_amount': total_amount,
        'customer': customer_name.title() if customer_name else "walk-in customer",
        'payment_method': payment_method if payment_method else "cash",
        'notes': notes,
        'sale_date': datetime.now().strftime("%Y-%m-%d"),
        'sale_time': datetime.now().strftime("%H:%M:%S")
    }
    
    # add to list using append method
    sales_records.append(sale_entry)
    
    print(f"\nsale recorded successfully!")
    print(f"sale id: {sale_entry['sale_id']}")
    print(f"total amount: ₹{total_amount:.2f}")

def view_all_sales():
    """display all sales records"""
    if not sales_records:
        print("\nno sales records found!")
        return
    
    print("\n" + "="*50)
    print(f"all sales records ({len(sales_records)} total)")
    print("="*50)
    
    for sale in sales_records:
        print(f"\nsale id: {sale['sale_id']}")
        print(f"product: {sale['product']}")
        print(f"quantity: {sale['quantity']} units")
        print(f"price per unit: ₹{sale['price_per_unit']:.2f}")
        print(f"total amount: ₹{sale['total_amount']:.2f}")
        print(f"customer: {sale['customer']}")
        print(f"payment: {sale['payment_method']}")
        print(f"date: {sale['sale_date']} at {sale['sale_time']}")
        if sale.get('notes'):
            print(f"notes: {sale['notes']}")1
        print("-" * 50)

def search_sales():
    """search sales by product name or customer name"""
    if not sales_records:
        print("\nno sales records to search!")
        return
    
    search_term = input("\nenter product name or customer name to search: ").strip().lower()
    
    # filter using list comprehension
    found_sales = [sale for sale in sales_records 
                   if search_term in sale['product'].lower() or 
                   search_term in sale['customer'].lower()]
    
    if not found_sales:
        print(f"\nno sales found matching '{search_term}'")
    else:
        print(f"\nfound {len(found_sales)} sale(s) matching '{search_term}':")
        print("="*50)
        for sale in found_sales:
            print(f"\nsale id: {sale['sale_id']} - {sale['product']}")
            print(f"quantity: {sale['quantity']} | total: ₹{sale['total_amount']:.2f}")
            print(f"customer: {sale['customer']} | date: {sale['sale_date']}")

def view_sales_statistics():
    """show sales statistics and analytics"""
    if not sales_records:
        print("\nno sales data available for statistics!")
        return
    
    print("\n" + "="*50)
    print("sales statistics and analytics")
    print("="*50)
    
    # calculate total revenue
    total_revenue = sum(sale['total_amount'] for sale in sales_records)
    total_units_sold = sum(sale['quantity'] for sale in sales_records)
    
    print(f"\ntotal sales: {len(sales_records)}")
    print(f"total revenue: ₹{total_revenue:.2f}")
    print(f"total units sold: {total_units_sold}")
    print(f"average sale value: ₹{total_revenue / len(sales_records):.2f}")
    
    # top selling products using dictionary
    product_sales = {}
    for sale in sales_records:
        product = sale['product']
        # dictionary get method with default value
        if product in product_sales:
            product_sales[product] += sale['quantity']
        else:
            product_sales[product] = sale['quantity']
    
    print("\n--- top selling products ---")
    # dictionary items method to get key-value pairs
    sorted_products = sorted(product_sales.items(), key=lambda x: x[1], reverse=True)
    for i, (product, quantity) in enumerate(sorted_products[:5], 1):
        print(f"{i}. {product}: {quantity} units")
    
    # revenue by product
    product_revenue = {}
    for sale in sales_records:
        product = sale['product']
        if product in product_revenue:
            product_revenue[product] += sale['total_amount']
        else:
            product_revenue[product] = sale['total_amount']
    
    print("\n--- top revenue generating products ---")
    sorted_revenue = sorted(product_revenue.items(), key=lambda x: x[1], reverse=True)
    for i, (product, revenue) in enumerate(sorted_revenue[:5], 1):
        print(f"{i}. {product}: ₹{revenue:.2f}")
    
    # payment method distribution
    payment_count = {}
    for sale in sales_records:
        method = sale['payment_method']
        payment_count[method] = payment_count.get(method, 0) + 1
    
    print("\n--- payment method distribution ---")
    for method, count in payment_count.items():
        percentage = (count / len(sales_records)) * 100
        print(f"{method}: {count} sales ({percentage:.1f}%)")
    
    # highest sale
    highest_sale = max(sales_records, key=lambda x: x['total_amount'])
    print(f"\n--- highest single sale ---")
    print(f"product: {highest_sale['product']}")
    print(f"amount: ₹{highest_sale['total_amount']:.2f}")
    print(f"date: {highest_sale['sale_date']}")

def view_daily_sales():
    """view sales for a specific date"""
    if not sales_records:
        print("\nno sales records available!")
        return
    
    date_input = input("\nenter date (yyyy-mm-dd) or press enter for today: ").strip()
    
    if not date_input:
        date_input = datetime.now().strftime("%Y-%m-%d")
    
    # filter sales by date
    daily_sales = [sale for sale in sales_records if sale['sale_date'] == date_input]
    
    if not daily_sales:
        print(f"\nno sales found for {date_input}")
    else:
        daily_revenue = sum(sale['total_amount'] for sale in daily_sales)
        print(f"\nsales for {date_input}:")
        print("="*50)
        print(f"total sales: {len(daily_sales)}")
        print(f"total revenue: ₹{daily_revenue:.2f}")
        print("\n--- sale details ---")
        for sale in daily_sales:
            print(f"\n{sale['product']} - ₹{sale['total_amount']:.2f}")
            print(f"customer: {sale['customer']} | time: {sale['sale_time']}")

def delete_sale():
    """delete a sale record by id"""
    if not sales_records:
        print("\nno sales records to delete!")
        return
    
    view_all_sales()
    
    try:
        sale_id = int(input("\nenter sale id to delete: "))
        
        # search for sale with matching id
        for i, sale in enumerate(sales_records):
            if sale['sale_id'] == sale_id:
                deleted = sales_records.pop(i)
                print(f"\ndeleted: {deleted['product']} sale of ₹{deleted['total_amount']:.2f}")
                return
        
        print(f"\nno sale found with id {sale_id}")
        
    except ValueError:
        print("\ninvalid input! please enter a valid sale id.")

def display_menu():
    """display the main menu options"""
    print("\n" + "="*50)
    print("sales management system")
    print("manage your sales efficiently!")
    print("="*50)
    print("\n1. add new sale")
    print("2. view all sales")
    print("3. search sales")
    print("4. view sales statistics")
    print("5. view daily sales")
    print("6. delete a sale record")
    print("7. save & exit")
    print("\n" + "="*50)

def main():
    """main program loop"""
    print("\nwelcome to sales management system!")
    print("track and manage all your sales transactions.\n")
    
    load_data_from_file()
    
    while True:
        display_menu()
        choice = input("\nenter your choice (1-7): ").strip()
        
        if choice == '1':
            add_sale_record()
        elif choice == '2':
            view_all_sales()
        elif choice == '3':
            search_sales()
        elif choice == '4':
            view_sales_statistics()
        elif choice == '5':
            view_daily_sales()
        elif choice == '6':
            delete_sale()
        elif choice == '7':
            save_data_to_file()
            print("\nthank you for using sales management system!")
            break
        else:
            print("\ninvalid choice! please select 1-7.")
        
        input("\npress enter to continue...")

# run the program
if __name__ == "__main__":
    main()


welcome to sales management system!
track and manage all your sales transactions.


no previous sales data found. starting fresh!

sales management system
manage your sales efficiently!

1. add new sale
2. view all sales
3. search sales
4. view sales statistics
5. view daily sales
6. delete a sale record
7. save & exit




enter your choice (1-7):  1



add new sale



product name:  2
quantity sold:  3
price per unit (in rupees):  30
customer name:  KIRAN
payment method (cash/card/upi/other):  CARD
additional notes (optional):  NOTHING



sale recorded successfully!
sale id: 1
total amount: ₹90.00



press enter to continue... 3



sales management system
manage your sales efficiently!

1. add new sale
2. view all sales
3. search sales
4. view sales statistics
5. view daily sales
6. delete a sale record
7. save & exit




enter your choice (1-7):  7



sales data saved successfully!

thank you for using sales management system!
